In [2]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import base64
from bs4 import BeautifulSoup
import markdown
from sqlalchemy import create_engine
import json
import warnings
import re

load_dotenv()

CONFIG = {
    'github_token': os.getenv('GITHUB_TOKEN'),
	'readme_token' : os.getenv('README_TOKEN'),
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_NAME'),
    'charset': os.getenv('DB_CHARSET')
}

In [3]:
# GitHub API 토큰 설정 (GitHub에서 Personal Access Token 생성 필요)
github_token = CONFIG['github_token']  # 여기에 토큰 입력
headers = {'Authorization': f'token {github_token}'}

# 검색 키워드 설정
#keyword = ["robotics", "ROS", "robot arm", "robot", "amr"]
keyword = ["robotics"]

# GitHub API 엔드포인트
serch_url = 'https://api.github.com/search/repositories'

In [4]:
# Github API 사용해서 Repository 가져오는 함수
def fetch_repos(query, sort='stars', order='desc', per_page=100):
    """
    GitHub API를 사용하여 키워드에 해당하는 레포지토리를 가져옵니다.
    스타 수가 100개 이상이며, 설명(description)에 중국어 문자가 없는 레포만 포함합니다.
    
    :param query: 검색할 키워드
    :param sort: 정렬 기준 (예: 'stars', 'forks')
    :param order: 정렬 순서 ('asc' 또는 'desc')
    :param per_page: 페이지당 결과 수 (최대 100)
    :return: 조건을 만족하는 레포지토리 리스트
    """
    all_repos = []
    page = 1
    
    # serch_url과 headers가 정의되었는지 확인 (실제 구현 시 필요)
    if 'serch_url' not in globals() or 'headers' not in globals():
        print("Error: serch_url 또는 headers 변수가 정의되지 않았습니다.")
        return []

    while True:
        params = {
            'q': f'{query} stars:>=100',  # star 수 100개 이상 조건 추가
            'sort': sort,
            'order': order,
            'per_page': per_page,
            'page': page
        }
        
        response = requests.get(serch_url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            repos = data.get('items', []) # 'items' 키로 레포 리스트를 가져옵니다.
            
            if not repos:
                # 현재 페이지에 레포가 없으면 (모든 결과를 가져옴) 루프를 종료합니다.
                break
            
            # **수정된 부분: 각 레포지토리를 순회하며 처리**
            for repo in repos:
                # repo는 딕셔너리 형태의 개별 레포지토리 정보입니다.
                desc = repo.get("description") or "" 
                
                # 설명에 중국어 문자(유니코드 범위)가 없는지 확인
                if not re.search(r"[\u4e00-\u9fff]", desc):
                    all_repos.append(repo)
            
            # 페이지를 증가시킵니다.
            page += 1
                
            # GitHub API rate limit을 고려해 최대 1000개로 제한 (10페이지)
            # 1페이지당 100개 * 10페이지 = 1000개
            if page > 10:
                print(f"최대 {per_page * 10}개의 레포를 가져왔으므로 검색을 종료합니다.")
                break
        else:
            # API 호출 에러 처리
            print(f"Error: {response.status_code} - {response.text}")
            break
            
    return all_repos

In [5]:
# repository readme 추출 함수
def get_readme(owner, repo, token=None):
    url = f"https://api.github.com/repos/{owner}/{repo}/readme"
    headers = {"Accept": "application/vnd.github.v3+json"}
    if token :
        headers["Authorization"] = f"token {token}"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        data = response.json()
        content = base64.b64decode(data["content"].encode("utf-8")).decode("utf-8", errors="ignore")
        
        try:
            html = markdown.markdown(content)
            text = BeautifulSoup(html, "html.parser").get_text(separator="\n")
            return text.strip()
        except Exception as e:
            print(f"{owner}/{repo}: Markdown -> Text 변환 실패 {e})")
            return content
        
    except requests.exceptions.HTTPError as e:
        print(f"{owner}/{repo}: README 없음 ({response.status_code})")
        return "None"
    
    except Exception as e:
        print(f"{owner}/{repo} README 디코드 실패: {e}")
        return "None"

In [6]:
# repo 필요한 데이터 추출하는 함수
def get_repo_details(repo):
    return {
		'id': repo['id'],
        'full_name': repo['full_name'],
        'stargazers_count': repo['stargazers_count'],
        'forks_count': repo['forks_count'],
        'language': repo['language'],
        'created_at': repo['created_at'],
        'updated_at': repo['updated_at'],
        'open_issues_count': repo['open_issues_count'],
        'topics': repo.get('topics', []),
        'license': repo['license']['spdx_id'] if repo.get('license') else None,
        'owner': repo['owner']['login'],
        'description': repo['description'],
		'is_fork' : repo['fork'],
		'readme' : get_readme(repo["owner"]["login"], repo["name"], CONFIG['readme_token'])
    }

In [7]:
# 데이터프레임을 MySQL에 저장하는 함수

# 경고 메시지를 무시하도록 설정 (Pandas의 to_sql 관련 경고가 뜰 수 있음)
warnings.filterwarnings("ignore", category=UserWarning, module="pandas.io.sql")

def save_df_to_sql(df):
    """
    주어진 데이터프레임을 github_repos와 github_readmes 테이블에 나누어 저장합니다.
    """
    
    # --- 1. 사용자 설정 ---
    # 사용자의 MySQL 데이터베이스 정보로 수정하세요.
    DB_USER = CONFIG['user']       # DB 사용자 이름
    DB_PASS = CONFIG['password']   # DB 비밀번호
    DB_HOST = CONFIG['host']           # DB 호스트 (예: 127.0.0.1)
    DB_PORT = "3306"                # DB 포트 (기본값 3306)
    DB_NAME = CONFIG['database']  # DB 이름
    
    # 데이터베이스 연결 문자열 생성 (MySQL + PyMySQL)
    connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?charset=utf8mb4"
    
    try:
        engine = create_engine(connection_string)
    except ImportError:
        print("오류: 'pymysql' 라이브러리가 필요합니다. 'pip install pymysql'로 설치해주세요.")
        return
    except Exception as e:
        print(f"데이터베이스 연결 오류: {e}")
        return

    # --- 2. 데이터 전처리 ---
    print("데이터 전처리를 시작합니다...")
    
    # 원본 수정을 피하기 위해 데이터프레임 복사
    df_processed = df.copy()
    
    # 'created_at', 'updated_at' 칼럼을 datetime 객체로 변환 (오류 무시)
    df_processed['created_at'] = pd.to_datetime(df_processed['created_at'], errors='coerce')
    df_processed['updated_at'] = pd.to_datetime(df_processed['updated_at'], errors='coerce')
    
    # 'topics' 칼럼이 리스트인 경우 JSON 문자열로 변환
    def serialize_topics(topics_list):
        if isinstance(topics_list, (list, dict)):
            return json.dumps(topics_list)
        elif pd.isna(topics_list):
            return None  # NaN, None 등은 SQL NULL로 처리
        return str(topics_list) # 이미 문자열이거나 다른 타입이면 문자열로
        
    if 'topics' in df_processed.columns:
        df_processed['topics'] = df_processed['topics'].apply(serialize_topics)
    
    print("데이터 전처리 완료.")

    # --- 3. 데이터 분리 ---
    
    # 'github_repos' 테이블에 필요한 칼럼 목록
    repo_columns = [
        'id', 'full_name', 'stargazers_count', 'forks_count', 'language',
        'created_at', 'updated_at', 'open_issues_count', 'topics',
        'license', 'owner', 'description', 'is_fork'
    ]
    
    # 'github_readmes' 테이블에 필요한 칼럼 목록
    readme_columns = ['full_name', 'readme']
    
    # DataFrame에 해당 칼럼들이 있는지 확인하고 분리
    # 존재하지 않는 칼럼이 있어도 오류 없이 진행하도록 처리
    valid_repo_cols = [col for col in repo_columns if col in df_processed.columns]
    valid_readme_cols = [col for col in readme_columns if col in df_processed.columns]
    
    df_repos = df_processed[valid_repo_cols]
    df_readmes = df_processed[valid_readme_cols]
    
    # 'github_readmes'의 'readme' 칼럼을 SQL 스키마에 맞게 'readme_content'로 변경
    if 'readme' in df_readmes.columns:
        df_readmes = df_readmes.rename(columns={'readme': 'readme_content'})

    # --- 4. 데이터베이스에 저장 (트랜잭션 사용) ---
    print("데이터베이스 저장을 시작합니다...")
    
    # 'with engine.begin()'을 사용하면 트랜잭션이 자동으로 관리됩니다.
    # 블록 내 코드가 모두 성공하면 커밋(commit), 오류 발생 시 롤백(rollback)됩니다.
    try:
        with engine.begin() as connection:
            
            # 1. 'github_repos' 테이블에 저장 (먼저 실행되어야 함)
            print("'github_repos' 테이블에 데이터를 저장합니다...")
            df_repos.to_sql(
                name='github_repos',
                con=connection,
                if_exists='append',    # 기존 테이블에 데이터 추가
                index=False,           # pandas 인덱스는 DB에 저장하지 않음
                chunksize=1000         # 대용량 데이터는 나눠서 삽입
            )
            print(f"{len(df_repos)}개의 레코드를 'github_repos'에 저장 시도 완료.")

            # 2. 'github_readmes' 테이블에 저장 (외래 키 제약 조건)
            # id와 collected_at은 AUTO_INCREMENT와 DEFAULT로 자동 생성되므로 제외
            print("'github_readmes' 테이블에 데이터를 저장합니다...")
            df_readmes.to_sql(
                name='github_readmes',
                con=connection,
                if_exists='append',
                index=False,
                chunksize=1000
            )
            print(f"{len(df_readmes)}개의 레코드를 'github_readmes'에 저장 시도 완료.")
        
        print("\n🎉 모든 데이터가 성공적으로 저장되었습니다. (트랜잭션 커밋)")

    except Exception as e:
        print(f"\n❌ 오류가 발생하여 모든 변경사항이 롤백되었습니다.")
        print(f"오류 상세 내용: {e}")
        print("팁: 'UNIQUE' 제약 조건(id 또는 full_name) 위반일 수 있습니다. 중복된 데이터를 확인해보세요.")

In [8]:
all_repos = []
for kw in keyword:
    print(f"Searching keyword: {kw}")
    result = fetch_repos(kw)
    all_repos.extend(result)

# full_name 기준 중복 제거
unique_repos = {repo["full_name"]: repo for repo in all_repos}
repos_list = list(unique_repos.values())
print(f"총 {len(repos_list)}개의 고유 레포 수집 완료")

# DataFrame 생성 및 CSV 저장
df = pd.DataFrame([get_repo_details(repo) for repo in repos_list])
df.to_csv('repos_data.csv', index=False, encoding='utf-8')
print("CSV 파일 저장 완료")

Searching keyword: robotics
최대 1000개의 레포를 가져왔으므로 검색을 종료합니다.
총 976개의 고유 레포 수집 완료
andrewkirillov/AForge.NET: README 없음 (404)
AgibotTech/agibot_x1_hardware: README 없음 (404)
MMehrez/MPC-and-MHE-implementation-in-MATLAB-using-Casadi: README 없음 (404)
EPFLXplore/XRE_LeggedRobot_HW: README 없음 (404)
CSV 파일 저장 완료


In [26]:
save_df_to_sql(df)

데이터 전처리를 시작합니다...
데이터 전처리 완료.
데이터베이스 저장을 시작합니다...
'github_repos' 테이블에 데이터를 저장합니다...
976개의 레코드를 'github_repos'에 저장 시도 완료.
'github_readmes' 테이블에 데이터를 저장합니다...
976개의 레코드를 'github_readmes'에 저장 시도 완료.

🎉 모든 데이터가 성공적으로 저장되었습니다. (트랜잭션 커밋)


In [23]:
df.head(5)

,id,full_name,stargazers_count,forks_count,language,created_at,updated_at,open_issues_count,topics,license,owner,description,is_fork,readme
0,71583602,Developer-Y/cs-video-courses,70098,9412,None,2016-10-21T17:02:11Z,2025-10-24T01:34:44Z,2,"[algorithms, bioinformatics, computational-bio...",None,Developer-Y,List of Computer Science courses with video le...,False,Computer Science courses with video lectures\n...
1,74627617,commaai/openpilot,58507,10348,Python,2016-11-24T01:33:30Z,2025-10-24T01:25:36Z,189,"[advanced-driver-assistance-systems, driver-as...",MIT,commaai,openpilot is an operating system for robotics....,False,openpilot\n\n\n\n\nopenpilot is an operating s...
2,615869301,mudler/LocalAI,35990,2855,Go,2023-03-18T22:58:02Z,2025-10-24T01:13:41Z,289,"[ai, api, audio-generation, decentralized, dis...",MIT,mudler,":robot: The free, Open Source alternative to O...",False,:bulb: Get help - \n❓FAQ\n \n💭Discussions\n \n...
3,712225112,Genesis-Embodied-AI/Genesis,27443,2520,Python,2023-10-31T03:33:11Z,2025-10-24T01:28:03Z,115,[],Apache-2.0,Genesis-Embodied-AI,A generative world for general-purpose robotic...,False,Genesis\n\n\n🔥 News\n\n\n\n\n[2025-08-05] Rele...
4,54376220,AtsushiSakai/PythonRobotics,26110,6911,Python,2016-03-21T09:34:43Z,2025-10-23T22:20:36Z,20,"[algorithm, animation, autonomous-driving, aut...",NOASSERTION,AtsushiSakai,Python sample codes and textbook for robotics ...,False,PythonRobotics\n\n\n\n\n\n\n\n\n\n\nPython cod...
